In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import matplotlib as mp
import scipy
import scipy.stats
import tensorflow as tf
#import tensorflow_hub as hub
import json
import pickle
import urllib
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer

print(tf.__version__)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
from wordcloud import WordCloud, STOPWORDS

In [ ]:
path = os.path.join(dirname, filename)
df = pd.read_csv(path) 

In [ ]:
df.columns

In [ ]:
data=df.tweet

In [ ]:
df.head()

In [ ]:
wc=WordCloud(width=200,height=100,background_color='black',stopwords=STOPWORDS
            ).generate(str(data))

## Main wordcloud

In [ ]:
fig=plt.figure(figsize=(10,10),facecolor='k',edgecolor='w')
plt.imshow(wc,interpolation='bicubic')
plt.axis("off")
plt.tight_layout()
plt.show()

In [ ]:
fig=plt.figure(figsize=(15,15),facecolor='k',edgecolor='w')
plt.imshow(wc,interpolation='bicubic')
plt.axis("off")
plt.tight_layout()
plt.show()

In [ ]:
from nltk.corpus import stopwords

from nltk.stem.wordnet import WordNetLemmatizer

import string
import gensim
from gensim import corpora

## Clean and Lemmatize into a document

In [ ]:
stop = set(stopwords.words('english'))

exclude = set(string.punctuation)

lemmma= WordNetLemmatizer() #base word conversion for bbetter tuning and performance

def clean(doc):
    stop_free=" ".join([i for i in doc.lower().split() if i not in stop])
    punc_free="".join([char for char in stop_free if char not in exclude])
    normalisation = " ".join(lemmma.lemmatize(word) for word in punc_free.split(' '))
    return normalisation

document=df.tweet.to_list()


doc_clean=[clean(docu).split() for docu in document ]

doc_clean[:10]

In [ ]:
df['tweet_clean']=pd.Series(doc_clean)


In [ ]:
df.head()

In [ ]:
dictionary=corpora.Dictionary(doc_clean)

print(dictionary)

In [ ]:
doc_word_freqcies=[dictionary.doc2bow(term) for term in doc_clean]
doc_word_freqcies[:30]

In [ ]:
from gensim.models import LdaModel

## Make LDA Model

In [ ]:

model=LdaModel(doc_word_freqcies,num_topics=9,id2word=dictionary,passes=400) 

In [ ]:
types= model.show_topics()
for t in types:
    print(t)
    print('----------------')

In [ ]:
diction={}
for i in range(6):
    words=model.show_topic(i,topn=20)
    #print(words)
    diction["Topic number" + "{}".format(i)]=[i[0] for i in words]
    
    
pd.DataFrame(diction)

In [ ]:
import pyLDAvis.gensim

In [ ]:
Vis=pyLDAvis.gensim.prepare(model,doc_word_freqcies,dictionary,sort_topics=False)

## Visualize topics

In [ ]:
pyLDAvis.display(Vis)

In [ ]:
# Create Corpus: Term Document Frequency
corpus = [dictionary.doc2bow(text) for text in doc_clean]


In [ ]:


def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=model, corpus=corpus, texts=doc_clean)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(10)

In [ ]:
vc=df_dominant_topic.Dominant_Topic.value_counts()
vc

In [ ]:
#topic 2 and 4  is 
#topic 3 is
#topic 5 IS 
#topic 1 is
dic={1.0:"2",2.0:"3",3.0:"4",4.0:"5",5.0:"6",6.0:'7',0.0:"1",8.0:'9',7.0:'8'}
vc=df_dominant_topic.Dominant_Topic.value_counts()

In [ ]:
dt=df_dominant_topic[["Dominant_Topic"]]

In [ ]:
dt

In [ ]:
dt.Dominant_Topic=dt.Dominant_Topic.apply(lambda row: dic[row])

In [ ]:
pd.DataFrame(dt.Dominant_Topic.value_counts())

## topic classification topic number and its meaning

## 4 racism
## 6 education
## 5 gun
## 8 health care
## 7 and 2 campaign
## 3  trump
## 9 biden
## 1 positive stuff




## He does talk about stuff related to 
## jobs  too, 
## depending on number of passes 
## and number of topics 
## we might get different results
## and classifications

## Topic word cloud

In [ ]:
# lda is assumed to be the variable holding the LdaModel object
import matplotlib.pyplot as plt
for t in range(model.num_topics):
    plt.figure()
#   plt.imshow(WordCloud().fit_words(model.show_topic(t, 200)))
    plt.imshow(WordCloud().fit_words(dict(model.show_topic(t, 200))))
    plt.axis("off")
    plt.title("Topic #" + str(t))
    plt.show()

In [ ]:
from collections import OrderedDict

data_lda = {i: OrderedDict(model.show_topic(i,25)) for i in range(6)}
#data_lda

In [ ]:
import pandas as pd

df_lda = pd.DataFrame(data_lda)
print(df_lda.shape)
df_lda = df_lda.fillna(0).T
print(df_lda.shape)

## Word heatmap

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

g=sns.clustermap(df_lda.corr(), center=0, cmap="RdBu", metric='cosine'
                 , linewidths=.75, figsize=(10, 10))
plt.setp(g.ax_heatmap.yaxis.get_majorticklabels(), rotation=0)
plt.show()
#plt.setp(ax_heatmap.get_yticklabels(), rotation=0)  # For y axis



## Sentiment analysis

In [ ]:
from textblob import TextBlob
df['pop']=df.tweet_clean.apply(lambda tw: TextBlob(' '.join(tw)).sentiment.polarity)
df

In [ ]:
df['sent']=df['pop'].apply(lambda p: 'positive' if p>0 else  
                           ( 'negative' if p<0  else 'neutral'))

In [ ]:
df

In [ ]:
plt.figure(figsize=(10, 10))
df['Time']=pd.to_datetime(df.timestamp)
sns.scatterplot(x=df['Time'], y=df['pop'],data=df,hue=df['sent']);




In [ ]:
df['year']=df.Time.apply(lambda x:x.year)
df.groupby('year')['pop'].describe()['mean'].plot(figsize=(10,10))

In [ ]:
print(df.sent.value_counts())
sns.countplot(x='sent', data = df);

In [ ]:
plt.figure(figsize=(10,6))
sns.distplot(df['pop'], bins=30)
plt.title('Sentiment Distribution',size = 10)
plt.xlabel('Polarity',size = 10)
plt.ylabel('Frequency',size = 10)
plt.show();

In [ ]:
count = df.groupby(['year','sent'])['tweet_clean'].count().reset_index().rename(
    columns={'tweet_clean':'count'})
count

In [ ]:
times = count.year.unique()
neut=[]
for year in count.year.unique():
    a=count[count['year']==year]
    if len(list(a[a['sent']=='neutral'].sent))>0:#list(a['sent'].unique()):
        c=list(a[a['sent']=='neutral']['count'])[0]
        neut.append(c)
    else:
        neut.append(0)
pos = count.loc[count['sent'] == 'positive']['count'].reset_index(drop = True)

neg=[]
for year in count.year.unique():
    a=count[count['year']==year]
    if len(list(a[a['sent']=='negative'].sent))>0:#list(a['sent'].unique()):
        c=list(a[a['sent']=='negative']['count'])[0]
        neg.append(c)
    else:
        neg.append(0)                        
plt.figure(figsize=(10,10))
plt.xticks(rotation='45')
lin1=plt.plot(times, pos, 'ro-', label='positive')
lin2=plt.plot(times, neut, 'g^-', label='neutral')
lin3=plt.plot(times, neg, 'b--', label='negative')
plt.legend()
plt.show

In [ ]:
df_dominant_topic.Dominant_Topic.shape

In [ ]:
df['topic']=df_dominant_topic.Dominant_Topic

In [ ]:
df.columns

## Make new nlp subset

In [ ]:
df_nlp=df[['id','Time','year','topic','pop','sent']]

In [ ]:
print(df_nlp.topic.value_counts())
sns.countplot(x='topic', data = df_nlp);

In [ ]:
import plotly.graph_objs as go
labels =['positive stuff','campaign','trump','racism','trump','gun','education','campaign','health care','biden']
outer_values = df_nlp.topic.value_counts()
inner_values=pd.Series()
for top in df_nlp.topic.unique():
    df_top=df_nlp[df_nlp['topic']==top]
    inner_values=inner_values.append(df_top[df_top['sent']=='positive'].value_counts())
    inner_values=inner_values.append(df_top[df_top['sent']=='negative'].value_counts())
    inner_values=inner_values.append(df_top[df_top['sent']=='neutral'].value_counts())
trace1 = go.Pie(
    hole=0.5,
    sort=False,
    direction='clockwise',
    domain={'x': [0.15, 0.85], 'y': [0.15, 0.85]},
    values=inner_values,
    textinfo='label',
    textposition='inside',
    marker={'line': {'color': 'white', 'width': 1}}
)
trace2 = go.Pie(
    hole=0.7,
    sort=False,
    direction='clockwise',
    values=outer_values,
    labels=labels,
    textinfo='label',
    textposition='outside',
    marker={'colors': ['green', 'red', 'blue'],
            'line': {'color': 'white', 'width': 1}}
)
fig = go.FigureWidget(data=[trace1, trace2])
fig



In [ ]:
df_nlp.head()

In [ ]:
df_nlp['hour']=df_nlp.Time.apply(lambda x:x.hour)

#What time he posts relates with the pop score?

df_nlp.groupby('hour')['pop'].describe()['mean'].plot(figsize=(10,10))

## Time of tweet and topic/sentiment related? i.e is Biden timewise-moody?

In [ ]:
sns.heatmap(df_nlp.corr())

## Biden ain't timewise-moody hmmm :-(
## But hey too many posts between 11pm-1am

In [ ]:
df_nlp.hour.value_counts()


## Now we have dived real deep. What else can we check here?